In [ ]:
# Movie recommender for the netflix data. Recommneds based on 
# keyword similarities. treats people(director and cast) as a single word token
# Try to get more plot description to improve accurracy
# May want to bump up the director/cast in the ratings

! pip install rake-nltk



In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

### Extract CSVs into DataFrames

In [14]:
#create a file from the netflix data
netflix_file = "netflix_titles.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.head()



,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [15]:
# turn director and names into lowercase, single words to avoid mixing similar names
netflix_df['director'] = netflix_df['director'].str.lower()
netflix_df['cast'] = netflix_df['cast'].str.lower()
netflix_df['director'] = netflix_df['director'].str.replace(" ","")
netflix_df['cast'] = netflix_df['cast'].str.replace(" ","")
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,jandinoasporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"petercullen,sumaleemontano,frankwelker,jeffrey...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"willfriedle,darrencriss,constancezimmer,kharyp...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,fernandolebrija,"nestacooper,katewalsh,johnmichaelhiggins,keith...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [18]:
netflix_df['type'].value_counts()

Movie      4265
TV Show    1969
Name: type, dtype: int64

In [17]:
#checking for duplicates and checking the total data
netflix_df.drop_duplicates()
netflix_df['type'].value_counts()

Movie      4265
TV Show    1969
Name: type, dtype: int64

In [20]:
# Take the key words from the plot description

# initializing the new column
netflix_df['Key_words'] = ""

for index, row in netflix_df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    r.extract_keywords_from_text(plot)
    words = r.get_ranked_phrases()
    # extracting the words by passing the text
    # getting the dictionary whith key words as keys and their scores as values
#     key_words_dict_scores = r.get_word_degrees()
    netflix_df.loc[index,'Key_words']=str(words)

In [21]:
# create a short dataframe that's easier to work with
short_df = netflix_df[["Key_words","title","director","cast"]]
# get rid of NaN
netflix_df = short_df.replace(np.nan, '', regex=True)

In [22]:
# remove symbols from key words list
netflix_df['Key_words'] = netflix_df['Key_words'].str.replace("[","")
netflix_df['Key_words'] = netflix_df['Key_words'].str.replace("]","")
netflix_df['Key_words'] = netflix_df['Key_words'].str.replace("'","")
netflix_df.head()

,Key_words,title,director,cast
0,"polar bear king must take back, evil archaeolo...",Norm of the North: King Sized Adventure,"richardfinn,timmaltby","alanmarriott,andrewtoth,briandobson,colehoward..."
1,"jandino asporaat riffs, rousing rendition, rai...",Jandino: Whatever it Takes,,jandinoasporaat
2,"three human allies, protect earth, onslaught, ...",Transformers Prime,,"petercullen,sumaleemontano,frankwelker,jeffrey..."
3,"prison ship crash unleashes hundreds, new auto...",Transformers: Robots in Disguise,,"willfriedle,darrencriss,constancezimmer,kharyp..."
4,"nerdy high schooler dani finally attracts, soc...",#realityhigh,fernandolebrija,"nestacooper,katewalsh,johnmichaelhiggins,keith..."


In [23]:
# combine key words, cast and director and set index as title
netflix_df['bag_of_words'] = netflix_df["Key_words"] + netflix_df["director"]
+ netflix_df["cast"] 
data = netflix_df[["title", "bag_of_words"]].set_index("title")
data

,bag_of_words
title,
Norm of the North: King Sized Adventure,"polar bear king must take back, evil archaeolo..."
Jandino: Whatever it Takes,"jandino asporaat riffs, rousing rendition, rai..."
Transformers Prime,"three human allies, protect earth, onslaught, ..."
Transformers: Robots in Disguise,"prison ship crash unleashes hundreds, new auto..."
#realityhigh,"nerdy high schooler dani finally attracts, soc..."
...,...
Red vs. Blue,"person shooter games, fiction films centers, c..."
Maron,"interviews fellow comedians, marc maron stars,..."
Little Baby Bum: Nursery Rhyme Friends,"playful animation engage, original music, nurs..."


In [24]:
# vectorize the key words
count = CountVectorizer()
count_matrix = count.fit_transform(data['bag_of_words'])

# generate the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [25]:

# create a Series for the movie titles so they are associated to an ordered numerical
# list used in the function to match the indexes
indices = pd.Series(data.index)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(data.index)[i])
        
    return recommended_movies
#  test the recommendation function
recommendations("Eat Pray Love")

['Cinta 100KG',
 "Larry Charles' Dangerous World of Comedy",
 'Can We Get Married?',
 "Murphy's Law of Love",
 'Incir Receli 2',
 'Love Family',
 "The Time Traveler's Wife",
 'Under the Eiffel Tower',
 'NiNoKuni',
 'Carmen Sandiego']

In [ ]:
# alternative text for removing key words. list function doesn't work
for index, row in netflix_df.iterrows():
    plot = row['description']
    a = r.extract_keywords_from_text(plot)
    b=r.get_ranked_phrases()
    key_words_dict_scores = r.get_word_degrees()
#     print(key_words_dict_scores)

row['Key_words'] = list(key_words_dict_scores.keys())
netflix_df

In [ ]:
netflix_df.drop_duplicates()
netflix_df["type"].value_counts()

In [ ]:


# initializing the new column
netflix_df['Key_words'] = ""

for index, row in netflix_df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
# df.drop(columns = ['Plot'], inplace = True)

In [ ]:
imdb_file = "IMDb movies.csv"
imdb_df = pd.read_csv(imdb_file)
imdb_df.head()

### Transform premise DataFrame

### Transform county DataFrame

### Create database connection

In [ ]:

rds_connection_string = "postgres:1117@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Confirm tables
engine.table_names()


### Load DataFrames into database

In [ ]:
new_county_df.to_sql(name='county', con=engine, if_exists='append', index=False)

In [ ]:
new_premise_df.to_sql(name='premise', con=engine, if_exists='append', index=False)